# Azure AI Agent with File Search Example

This notebook demonstrates how to create an Azure AI agent that uses a file search tool to answer user questions based on uploaded documents.

## Features Covered:
- File upload and management
- Vector store creation and management
- File search tool configuration
- Document-based question answering
- Resource cleanup and management

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

In [ ]:
#!about
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.*"
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.*"
#r "nuget: dotenv.net"

using Azure.Identity;
using Microsoft.Agents.AI;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Microsoft.Extensions.AI;
using dotenv.net;
using System.IO;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Text.Json;
using OpenAI.Assistants;
using OpenAI.Responses;
using OpenAI;
using OpenAI.Files;
using OpenAI.VectorStores;
using System.ClientModel;


## Define Sample Queries

Let's define some sample questions to ask about the uploaded document:

In [ ]:
var userInputs = new string[]
{
    "Who is the youngest employee?",
    "Who works in sales?",
    "I have a customer request, who can help me?",
};

## Main File Search Example

This example demonstrates the complete workflow:
1. Upload a file
2. Create a vector store
3. Create a file search tool
4. Create an agent with file search capabilities
5. Query the agent about the document content

In [ ]:
#pragma warning disable CA1812
#pragma warning disable MEAI001
#pragma warning disable OPENAI001
/// <summary>
/// Azure AI Agent with File Search (RAG) Example using Microsoft Agent Framework.
/// This example demonstrates how to create an agent with built-in RAG capabilities
/// using Foundry service's file search tool.
/// </summary>
async Task MainFileSearchExampleAsync()
{
    Console.WriteLine("=== Azure AI Agent with File Search Example ===\n");

    // Sample user inputs to query the document
    var userInputs = new[]
    {
        "What employees are listed in the document?",
        "Tell me about the Engineering department.",
        "Who is the manager of the Sales team?"
    };

    OpenAIFile? uploadedFile = null;
    VectorStore? vectorStore = null;
    AIAgent? agent = null;
    // Get configuration from environment variables
    var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
    var endpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
    var deploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
    try
    {
        // Create an AI Project client and get an OpenAI client that works with the Foundry service
        AIProjectClient aiProjectClient = new(new Uri(endpoint), new AzureCliCredential());
        OpenAIClient openAIClient = aiProjectClient.GetProjectOpenAIClient();
        Console.WriteLine("✅ Client initialized");

        // Check if the employees file exists
        var filePath = Path.Combine(".", "resources", "employees.txt");
        var absolutePath = Path.GetFullPath(filePath);

        if (!File.Exists(absolutePath))
        {
            Console.WriteLine($"❌ File not found at: {absolutePath}");
            Console.WriteLine("Please ensure the employees.pdf file is placed in the 'resources' directory.");
            return;
        }

        Console.WriteLine($"📄 Using file: {absolutePath}");

        // Upload the file that contains the data to be used for RAG to the Foundry service
        OpenAIFileClient fileClient = openAIClient.GetOpenAIFileClient();
        ClientResult<OpenAIFile> uploadResult = await fileClient.UploadFileAsync(
            filePath: absolutePath,
            purpose: FileUploadPurpose.Assistants);
        uploadedFile = uploadResult.Value;
        Console.WriteLine($"✅ Uploaded file, file ID: {uploadedFile.Id}");

        // Create a vector store in the Foundry service using the uploaded file
        VectorStoreClient vectorStoreClient = openAIClient.GetVectorStoreClient();
        ClientResult<VectorStore> vectorStoreResult = await vectorStoreClient.CreateVectorStoreAsync(
            options: new VectorStoreCreationOptions()
            {
                Name = "employees-knowledge-base",
                FileIds = { uploadedFile.Id }
            });
        vectorStore = vectorStoreResult.Value;
        Console.WriteLine($"✅ Created vector store, ID: {vectorStore.Id}");

        // Wait for vector store to be ready
        while (vectorStore.Status == VectorStoreStatus.InProgress)
        {
            await Task.Delay(1000);
            var updatedStore = await vectorStoreClient.GetVectorStoreAsync(vectorStore.Id);
            vectorStore = updatedStore.Value;
        }
        Console.WriteLine($"✅ Vector store status: {vectorStore.Status}");

        // Create a file search tool with the vector store
        var fileSearchTool = new HostedFileSearchTool()
        {
            Inputs = [new HostedVectorStoreContent(vectorStore.Id)]
        };

        // Create an agent with file search capabilities
        agent = await aiProjectClient.CreateAIAgentAsync(
            model: deploymentName,
            name: "EmployeeSearchAgent",
            instructions: "You are a helpful assistant that can search through the employee directory to answer questions about employees. Answer questions using the provided context and cite specific details when available.",
            tools: [fileSearchTool]);
        Console.WriteLine($"✅ Created agent: {agent.Name}");

        // Create a thread for multi-turn conversation
        AgentThread thread = agent.GetNewThread();

        // Query the agent with the user inputs
        Console.WriteLine("\n=== Querying the Document ===");

        foreach (var userInput in userInputs)
        {
            Console.WriteLine($"\n🤔 User: '{userInput}'");
            var response = await agent.RunAsync(userInput, thread);
            Console.WriteLine($"🤖 Agent: {response}");
        }

        Console.WriteLine("\n=== Resource Information ===");
        Console.WriteLine($"📋 Vector store: {vectorStore.Id}");
        Console.WriteLine($"📄 File: {uploadedFile.Id}");
        Console.WriteLine($"🤖 Agent: {agent.Name}");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error in main execution: {ex.Message}");
        Console.WriteLine($"   Stack trace: {ex.StackTrace}");
    }
    finally
    {
        // Cleanup resources
        Console.WriteLine("\n=== Cleaning up resources ===");
        try
        {
            if (endpoint != null)
            {
                AIProjectClient aiProjectClient = new(new Uri(endpoint), new AzureCliCredential());
                OpenAIClient openAIClient = aiProjectClient.GetProjectOpenAIClient();

                if (uploadedFile != null)
                {
                    OpenAIFileClient fileClient = openAIClient.GetOpenAIFileClient();
                    await fileClient.DeleteFileAsync(uploadedFile.Id);
                    Console.WriteLine($"🗑️  Deleted file: {uploadedFile.Id}");
                }

                if (vectorStore != null)
                {
                    VectorStoreClient vectorStoreClient = openAIClient.GetVectorStoreClient();
                    await vectorStoreClient.DeleteVectorStoreAsync(vectorStore.Id);
                    Console.WriteLine($"🗑️  Deleted vector store: {vectorStore.Id}");
                }

                if (agent != null)
                {
                    await aiProjectClient.Agents.DeleteAgentAsync(agent.Name);
                    Console.WriteLine($"🗑️  Deleted agent: {agent.Name}");
                }
            }
        }
        catch (Exception cleanupEx)
        {
            Console.WriteLine($"⚠️ Cleanup error: {cleanupEx.Message}");
        }
    }
}


## Execute the Example

Run the main function to see file search in action:

In [ ]:
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".", "..", "..","..", ".env") })); 
// Run the main example
await MainFileSearchExampleAsync();

## Key Takeaways

1. **File Upload**: Documents must be uploaded to Azure AI before they can be searched
2. **Vector Stores**: Files are organized in vector stores for efficient searching
3. **File Search Tool**: The `HostedFileSearchTool` enables document-based question answering
4. **Resource Management**: Always clean up uploaded files and vector stores to avoid costs
5. **Multiple Files**: You can upload multiple documents to a single vector store
6. **Document Types**: Supports TXT, DOCX, and other common document formats

## Best Practices

1. **File Management**: Keep track of uploaded files and their purposes
2. **Error Handling**: Always handle upload and processing errors gracefully
3. **Resource Cleanup**: Use try-finally blocks to ensure resource cleanup
4. **Document Quality**: Ensure uploaded documents are well-formatted for better search results
5. **Query Optimization**: Frame questions clearly to get better search results

## Use Cases

- **Document Q&A**: Answer questions about uploaded documents
- **Knowledge Base**: Create searchable knowledge bases from documents
- **Research Assistant**: Help users find specific information in large document sets
- **Compliance**: Search through policy documents and regulations
- **Customer Support**: Search through product manuals and documentation